In [33]:
!pip install pdf2image
#!pip install opencv-python
#!pip install pillow
#!pip install pytesseract

In [ ]:
!pip install tesseract
!pip install tesseract-ocr

In [ ]:
# https://github.com/UB-Mannheim/tesseract/wiki

In [57]:
pytesseract.pytesseract.tesseract_cmd = 'C:/OCR/Tesseract/tesseract.exe' # надо менять, если путь другой

In [43]:
import cv2
import pytesseract
from pdf2image import convert_from_path

pages = convert_from_path('maestro_book1.pdf', 100, poppler_path=r"poppler-0.68.0\bin")

In [142]:
def clean(text, num):
    foot = 2 if num % 2 == 0 else 3
    return " ".join(x for x in text.replace("-\n", "").split("\n")[2:-foot])

In [175]:
page_text = []
from tqdm import tqdm

for i in tqdm(range(8, 410)):
    pages[i].save(f'out{i}.jpg', 'JPEG')
    imgcv = cv2.imread(f'out{i}.jpg')
    imgcv = cv2.cvtColor(imgcv, cv2.COLOR_BGR2RGB)
    text = pytesseract.image_to_string(imgcv,  lang='eng+rus')
    page_text.append(clean(text, i-1))
    #9-409

100%|████████████████████████████████████████████████████████████████████████████████| 402/402 [11:10<00:00,  1.67s/it]


In [176]:
full_text = "".join(x for x in page_text)

In [177]:
with open("maestro_book1.txt", "w", encoding="utf-8") as g:
    g.write(full_text)

In [ ]:
pages = convert_from_path('maestro_book2.pdf', 100, poppler_path=r"poppler-0.68.0\bin")

In [296]:
def clean2(text, num):
    header = 3 if num % 2 == 0 else 7
    text = " ".join(x for x in text.split()[header:] if x != "")
    text = re.sub(r"[\s]{2,}", "", text.replace("\xad", " "))
    return text

In [302]:
from PyPDF2 import PdfReader

reader = PdfReader("maestro_book2.pdf")
text = " ".join(x for x in reader.pages[8].extract_text().split()[2:])
text = re.sub(r"[\s]{2,}", "", text.replace("\xad", " "))# титул
for num in tqdm(range(9, 865)): # 865
    text += clean2(reader.pages[num].extract_text(), num-1)

100%|████████████████████████████████████████████████████████████████████████████████| 856/856 [01:12<00:00, 11.74it/s]


In [303]:
with open("maestro_book2.txt", "w", encoding="utf-8") as g:
    g.write(text)

In [304]:
len(full_text.split())

125716

In [305]:
len(text.split())

400963

In [ ]:
<div "author_page_grid_article__title author_page_article_title"

In [307]:
import urllib.request
import requests
import re
import os
from bs4 import BeautifulSoup

In [448]:
url = "https://vk.com/@evgenyponasenkov"

session = requests.session()
session.trust_env = False

req = session.get(url)
page = req.text
soup = BeautifulSoup(page, 'html.parser')

a = soup.find_all('a', href=True)

In [438]:
import pandas as pd

df = pd.DataFrame(columns=["url", "title", "date", "text"])

In [450]:
hrefs = ["https://vk.com"+i["href"] for i in a[3:-1]]

In [451]:
for href in tqdm(hrefs):
    req = session.get(href)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    text = " ".join(x.text for x in soup.find_all("p"))
    title = soup.find("h1").text
    # мне лень это всё в группы клеить
    date = re.findall(r".{4}-.{2}-.{2}", re.findall(r'"datePublished".*T', page)[0])[0]
    df = pd.concat([df, pd.DataFrame({"url": href, "title": title, "date": date, "text": text}, index=[0])])


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:03<00:00,  6.05it/s]


In [452]:
df

,url,title,date,text
0,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011),2021-02-24,"Начну, простите, с конца. Да, митинги 10-го и ..."
0,https://vk.com/@evgenyponasenkov-istorik-evgen...,Историк Евгений Понасенков: Россия - это ходяч...,2021-02-17,"Историк, режиссер, оперный певец и общественны..."
0,https://vk.com/@evgenyponasenkov-dva-gogolya-i...,"Два Гоголя, или Неизбежность поражения",2021-02-14,"Не секрет, что общество, эпоха, семья - да, чт..."
0,https://vk.com/@evgenyponasenkov-otprazdnuem-k...,Отпразднуем капитуляцию кремлевского гарнизона...,2021-01-27,"Начну не с того, что правительство России не в..."
0,https://vk.com/@evgenyponasenkov-gosudarstvenn...,Государственный переворот и «НТВшники» (10. 04...,2021-01-21,"Как это ни печально, но скоро основатель движе..."
0,https://vk.com/@evgenyponasenkov-1812-ubileiny...,"1812: юбилейный зомбоящик, верни дворянам масл...",2020-11-30,Превеликая кампания по зомбированию населения ...
0,https://vk.com/@evgenyponasenkov-1812-god-otve...,1812 год: ответ историка Патриарху Кириллу (10...,2020-11-26,В недавнем азартном отмечании юбилея войны 181...
0,https://vk.com/@evgenyponasenkov-evgenialnyi-s...,ЕвГениальный список архитекторов,2020-09-16,"Альбомы по всей архитектуре Древнего Египта, Г..."
0,https://vk.com/@evgenyponasenkov-intelligentsk...,Интеллигентский мазохизм — страшнее терроризма...,2020-08-17,"Господа-товарищи, всему же есть пределы! Вчера..."
0,https://vk.com/@evgenyponasenkov-chudovischnye...,Чудовищные уроки 1917 года: убийство как истор...,2020-08-09,"Надо признать, что в начале прошлого века коре..."


In [453]:
with open("maestro_book2.txt", "r", encoding="utf-8") as f:
    text = f.read()

df = pd.concat([df, pd.DataFrame({"url": "https://www.moscowbooks.ru/bookinist/book/308360/",
                                  "title": 'Первая научная история войны 1812 года',
                                  "date": 2018,
                                  "text": text}, index=[0])])

with open("maestro_book1.txt", "r", encoding="utf-8") as f:
    text = f.read()

df = pd.concat([df, pd.DataFrame({"url": "https://www.ozon.ru/product/pervaya-nauchnaya-istoriya-voyny-1812-goda-trete-izdanie-ponasenkov-evgeniy-nikolaevich-250460112/?sh=cAScFiM5SQ",
                                  "title": 'Правда о войне 1812 года',
                                  "date": 2004,
                                  "text": text}, index=[0])])



In [457]:
df.reset_index(drop=True).iloc[:-2].to_csv("maestro_vk.csv")

In [414]:
date = re.findall(r".{4}-.{2}-.{2}", re.findall(r'"datePublished".*T', str(a[0]))[0])

['2020-06-10']

In [534]:
df = pd.DataFrame(columns=["url", "title", "date", "text"])

In [535]:
for i in range(1, 2):

    url = f"https://www.mk.ru/blogs/blog-evgeniya-ponasenkova.html?page={i}"

    session = requests.session()
    session.trust_env = False

    req = session.get(url)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')

    a = soup.find_all('a', href=True)
    refs = []
    for i in a:
        href = i["href"]
        if "https://www.mk.ru/blogs/posts/" in href:
            refs.append(href)

    for href in refs[:-5]:  # 5 последних на всех страницах
        sub_req = session.get(href)
        sub_page = sub_req.text
        sub_soup = BeautifulSoup(sub_page, 'html.parser')
        text = " ".join(x.text for x in sub_soup.find_all("p"))
        text = text.replace("\n", " ").split()
        title = sub_soup.find("h1").text + ". Блог Евгения Понасенкова на mk.ru"
        date = " ".join(text[i] for i in range(3))
        text = " ".join(x for x in text[8:])  # до этого не важная информация
        df = pd.concat([df, pd.DataFrame({"url": href, "title": title, "date": date, "text": text}, index=[0])])

In [538]:
df.reset_index(drop=True).to_csv("maestro.blog.csv")